In [2]:
from keras.models import Sequential
from keras.layers import Dense
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

Using TensorFlow backend.


In [15]:
dataset = pd.read_csv('data/fullDF.csv', index_col=0)
dataset = dataset.drop(['GRID_TYPE', 'PLAYER_ID', 'SHOT_ATTEMPTED_FLAG', 'TEAM_ID'], axis=1)
dataset_copy = dataset

In [16]:
categorical_cols = ['ACTION_TYPE', 'HTM', 'PERIOD', 'PLAYER_NAME', 'QUARTER', 'SHOT_TYPE', 'SHOT_ZONE_AREA',
                    'SHOT_ZONE_BASIC', 'SHOT_ZONE_RANGE', 'TEAM_NAME', 'VTM']

def onehot_encode(data):
    for feature in categorical_cols:
        one_hot = pd.get_dummies(data[feature], prefix=feature)
        data = data.drop(feature, axis = 1)
        data = data.join(one_hot)
    return data

dataset = onehot_encode(dataset)

In [17]:
correlation_df = dataset.corr()[['SHOT_MADE_FLAG']]
correlation_df.rename(columns={"SHOT_MADE_FLAG": "correlation_with_making_shot"}, inplace=True)
correlation_df

,correlation_with_making_shot
EVENTTIME,0.012396
LOC_X,0.007481
LOC_Y,-0.036743
MINUTES_REMAINING,0.011510
SECONDS_REMAINING,0.010951
...,...
VTM_SAC,0.004892
VTM_SAS,0.003040
VTM_TOR,-0.004145
VTM_UTA,0.000401


In [18]:
correlation_df = correlation_df.sort_values(by='correlation_with_making_shot', ascending=False)

In [19]:
for i, j in correlation_df.iterrows():
    corr = j[0]
    if corr > 0.03 or corr < -0.03:
        print(i, j) 
        print() 

SHOT_MADE_FLAG correlation_with_making_shot    1.0
Name: SHOT_MADE_FLAG, dtype: float64

SHOT_ZONE_BASIC_Restricted Area correlation_with_making_shot    0.199887
Name: SHOT_ZONE_BASIC_Restricted Area, dtype: float64

SHOT_ZONE_RANGE_Less Than 8 ft. correlation_with_making_shot    0.167723
Name: SHOT_ZONE_RANGE_Less Than 8 ft., dtype: float64

SHOT_ZONE_AREA_Center(C) correlation_with_making_shot    0.133036
Name: SHOT_ZONE_AREA_Center(C), dtype: float64

PLAYER_AVG correlation_with_making_shot    0.120922
Name: PLAYER_AVG, dtype: float64

SHOT_TYPE_2PT Field Goal correlation_with_making_shot    0.12018
Name: SHOT_TYPE_2PT Field Goal, dtype: float64

ACTION_TYPE_Dunk Shot correlation_with_making_shot    0.10812
Name: ACTION_TYPE_Dunk Shot, dtype: float64

ACTION_TYPE_Cutting Layup Shot correlation_with_making_shot    0.085503
Name: ACTION_TYPE_Cutting Layup Shot, dtype: float64

ACTION_TYPE_Cutting Dunk Shot correlation_with_making_shot    0.082773
Name: ACTION_TYPE_Cutting Dunk Shot, d

Features with most correlation:

Shot Zone

Player Average

Shot Type

Action Type

Shot Distance

LOC_Y

In [23]:
dataset_imp_features_only = dataset_copy[['SHOT_MADE_FLAG', 'SHOT_ZONE_BASIC', 'SHOT_ZONE_AREA', 'SHOT_ZONE_RANGE',
                                    'PLAYER_AVG', 'SHOT_TYPE', 'ACTION_TYPE', 'SHOT_DISTANCE', 'LOC_Y']]
dataset_imp_features_only

,SHOT_MADE_FLAG,SHOT_ZONE_BASIC,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,PLAYER_AVG,SHOT_TYPE,ACTION_TYPE,SHOT_DISTANCE,LOC_Y
2732,0,Above the Break 3,Left Side Center(LC),24+ ft.,0.510288,3PT Field Goal,Jump Shot,25,125.2763
12130,1,Restricted Area,Center(C),Less Than 8 ft.,0.357466,2PT Field Goal,Running Dunk Shot,0,-5.5908
74191,1,Restricted Area,Center(C),Less Than 8 ft.,0.508380,2PT Field Goal,Hook Shot,2,78.5379
19621,0,Restricted Area,Center(C),Less Than 8 ft.,0.452555,2PT Field Goal,Layup Shot,1,238.1948
66474,0,Above the Break 3,Right Side Center(RC),24+ ft.,0.244444,3PT Field Goal,Jump Shot,25,139.8303
...,...,...,...,...,...,...,...,...,...
70944,0,Mid-Range,Right Side Center(RC),16-24 ft.,0.538462,2PT Field Goal,Jump Shot,18,217.1834
76301,0,Above the Break 3,Right Side Center(RC),24+ ft.,0.487013,3PT Field Goal,Running Pull-Up Jump Shot,25,206.2505
48761,0,In The Paint (Non-RA),Center(C),8-16 ft.,0.448649,2PT Field Goal,Jump Shot,12,246.5670
2308,0,Restricted Area,Center(C),Less Than 8 ft.,0.504292,2PT Field Goal,Layup Shot,1,50.1469


In [24]:
train_data, test_data = train_test_split(dataset, test_size=0.2)
x_train = train_data.drop(['SHOT_MADE_FLAG'], axis=1)
y_train = train_data[['SHOT_MADE_FLAG']]
x_test = test_data.drop(['SHOT_MADE_FLAG'], axis=1)
y_test = test_data[['SHOT_MADE_FLAG']]

categorical_cols1 = ['ACTION_TYPE', 'SHOT_TYPE', 'SHOT_ZONE_AREA', 'SHOT_ZONE_BASIC', 'SHOT_ZONE_RANGE']

def onehot_encode1(data):
    for feature in categorical_cols1:
        one_hot = pd.get_dummies(data[feature], prefix=feature)
        data = data.drop(feature, axis = 1)
        data = data.join(one_hot)
    return data

dataset_imp_features_only = onehot_encode1(dataset_imp_features_only)

train_data1, test_data1 = train_test_split(dataset_imp_features_only, test_size=0.2)
x_train1 = train_data1.drop(['SHOT_MADE_FLAG'], axis=1)
y_train1 = train_data1[['SHOT_MADE_FLAG']]
x_test1 = test_data1.drop(['SHOT_MADE_FLAG'], axis=1)
y_test1 = test_data1[['SHOT_MADE_FLAG']]

In [25]:
x_train.head()
x_train1.head()

,PLAYER_AVG,SHOT_DISTANCE,LOC_Y,ACTION_TYPE_Alley Oop Dunk Shot,ACTION_TYPE_Alley Oop Layup shot,ACTION_TYPE_Cutting Dunk Shot,ACTION_TYPE_Cutting Finger Roll Layup Shot,ACTION_TYPE_Cutting Layup Shot,ACTION_TYPE_Driving Bank Hook Shot,ACTION_TYPE_Driving Bank shot,...,SHOT_ZONE_BASIC_In The Paint (Non-RA),SHOT_ZONE_BASIC_Left Corner 3,SHOT_ZONE_BASIC_Mid-Range,SHOT_ZONE_BASIC_Restricted Area,SHOT_ZONE_BASIC_Right Corner 3,SHOT_ZONE_RANGE_16-24 ft.,SHOT_ZONE_RANGE_24+ ft.,SHOT_ZONE_RANGE_8-16 ft.,SHOT_ZONE_RANGE_Back Court Shot,SHOT_ZONE_RANGE_Less Than 8 ft.
39471,0.451220,17,88.0143,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0
926,0.527869,8,2.2868,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,1,0,0
54252,0.414430,9,284.6756,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,0
76642,0.503464,5,58.3177,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
71131,0.487013,22,31.9727,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0


In [27]:
model = Sequential()
model.add(Dense(1000, input_dim=len(x_train.columns), activation='relu'))
model.add(Dense(500, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(x_train, y_train, epochs=80, batch_size=1024)

Epoch 1/80
67573/67573 [==============================] - 6s 90us/step - loss: 2.5020 - accuracy: 0.5339
Epoch 2/80
67573/67573 [==============================] - 5s 71us/step - loss: 0.6734 - accuracy: 0.5923
Epoch 3/80
67573/67573 [==============================] - 5s 75us/step - loss: 0.6797 - accuracy: 0.5935
Epoch 4/80
67573/67573 [==============================] - 5s 71us/step - loss: 0.6704 - accuracy: 0.6058
Epoch 5/80
67573/67573 [==============================] - 5s 70us/step - loss: 0.6790 - accuracy: 0.5984
Epoch 6/80
67573/67573 [==============================] - 5s 75us/step - loss: 0.6748 - accuracy: 0.6075
Epoch 7/80
67573/67573 [==============================] - 5s 70us/step - loss: 0.6469 - accuracy: 0.6295
Epoch 8/80
67573/67573 [==============================] - 5s 78us/step - loss: 0.6586 - accuracy: 0.6227
Epoch 9/80
67573/67573 [==============================] - 7s 105us/step - loss: 0.6475 - accuracy: 0.6279
Epoch 10/80
67573/67573 [=============================

67573/67573 [==============================] - 5s 74us/step - loss: 0.4796 - accuracy: 0.7483
Epoch 79/80
67573/67573 [==============================] - 5s 77us/step - loss: 0.4691 - accuracy: 0.7555
Epoch 80/80
67573/67573 [==============================] - 5s 76us/step - loss: 0.4723 - accuracy: 0.7530


In [28]:
y_test_pred = model.predict_classes(x_test)

In [29]:
print(f"accuracy: {round(accuracy_score(y_test, y_test_pred), 3)}")

accuracy: 0.61


In [26]:
model1 = Sequential()
model1.add(Dense(150, input_dim=len(x_train1.columns), activation='relu'))
model1.add(Dense(75, activation='relu'))
model1.add(Dense(10, activation='relu'))
model1.add(Dense(1, activation='sigmoid'))

model1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model1.fit(x_train1, y_train1, epochs=80, batch_size=1024)

y_test_pred1 = model1.predict_classes(x_test1)
print(f"accuracy (only using most correlated features): {round(accuracy_score(y_test1, y_test_pred1), 3)}")

Epoch 1/80
67573/67573 [==============================] - 2s 28us/step - loss: 0.9311 - accuracy: 0.5542
Epoch 2/80
67573/67573 [==============================] - 1s 10us/step - loss: 0.6564 - accuracy: 0.6180
Epoch 3/80
67573/67573 [==============================] - 1s 10us/step - loss: 0.6472 - accuracy: 0.6325
Epoch 4/80
67573/67573 [==============================] - 1s 10us/step - loss: 0.6366 - accuracy: 0.6415
Epoch 5/80
67573/67573 [==============================] - 1s 11us/step - loss: 0.6347 - accuracy: 0.6423
Epoch 6/80
67573/67573 [==============================] - 1s 12us/step - loss: 0.6348 - accuracy: 0.6407
Epoch 7/80
67573/67573 [==============================] - 1s 12us/step - loss: 0.6314 - accuracy: 0.6426
Epoch 8/80
67573/67573 [==============================] - 1s 12us/step - loss: 0.6291 - accuracy: 0.6454
Epoch 9/80
67573/67573 [==============================] - 1s 11us/step - loss: 0.6292 - accuracy: 0.6457
Epoch 10/80
67573/67573 [==============================

67573/67573 [==============================] - 1s 12us/step - loss: 0.6195 - accuracy: 0.6513
Epoch 79/80
67573/67573 [==============================] - 1s 12us/step - loss: 0.6191 - accuracy: 0.6534
Epoch 80/80
67573/67573 [==============================] - 1s 11us/step - loss: 0.6193 - accuracy: 0.6526
accuracy (only using most correlated features): 0.645


In [100]:
model1 = Sequential()
model1.add(Dense(1000, input_dim=len(x_train.columns), activation='relu'))
model1.add(Dense(100, activation='relu'))
model1.add(Dense(1, activation='sigmoid'))

model1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model1.fit(x_train, y_train, epochs=120, batch_size=1024)

Epoch 1/120
67573/67573 [==============================] - 4s 65us/step - loss: 2.7159 - accuracy: 0.5378
Epoch 2/120
67573/67573 [==============================] - 3s 51us/step - loss: 0.6648 - accuracy: 0.6088
Epoch 3/120
67573/67573 [==============================] - 3s 51us/step - loss: 0.6679 - accuracy: 0.6107
Epoch 4/120
67573/67573 [==============================] - 3s 49us/step - loss: 0.6580 - accuracy: 0.6225
Epoch 5/120
67573/67573 [==============================] - 4s 56us/step - loss: 0.6785 - accuracy: 0.6100
Epoch 6/120
67573/67573 [==============================] - 4s 58us/step - loss: 0.6572 - accuracy: 0.6278
Epoch 7/120
67573/67573 [==============================] - 4s 58us/step - loss: 0.6516 - accuracy: 0.6307
Epoch 8/120
67573/67573 [==============================] - 4s 58us/step - loss: 0.6901 - accuracy: 0.6164
Epoch 9/120
67573/67573 [==============================] - 4s 62us/step - loss: 0.6457 - accuracy: 0.6353
Epoch 10/120
67573/67573 [====================

In [101]:
y_test_pred_m1 = model1.predict_classes(x_test)
print(f"model 1 accuracy: {round(accuracy_score(y_test, y_test_pred_m1), 3)}")

model 1 accuracy: 0.607
